In [5]:
import os

import numpy as np
import pandas as pd

from PIL import Image, ImageChops
from matplotlib import pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class Network_1(nn.Module):
    def __init__(self):
        super(Network_1, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, -1)
    
class Network_2(nn.Module):
    def __init__(self):
        super(Network_2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.drop_out1 = nn.Dropout2d(p=0.25)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)
        self.drop_out2 = nn.Dropout2d(p=0.25)
        
        self.fc1 = nn.Linear(576, 64)
        self.drop_out3 = nn.Dropout2d(p=0.25)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.drop_out1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)
        x = self.drop_out2(x)
        
        x = x.view(-1, 576)
        x = self.fc1(x)
        x = self.drop_out3(x)
        x = self.fc2(x)
        return F.log_softmax(x, -1)
    
class Network_3(nn.Module):
    def __init__(self):
        super(Network_3, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=7)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5)
        self.drop_out1 = nn.Dropout2d(p=0.25)
        
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv4 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3)
        self.drop_out2 = nn.Dropout2d(p=0.25)
        
        self.fc1 = nn.Linear(128, 32)
        self.drop_out3 = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.drop_out1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, 3)
        x = self.drop_out2(x)
        
        x = x.view(-1, 128)
        x = self.fc1(x)
        x = self.drop_out3(x)
        x = self.fc2(x)
        return F.log_softmax(x, -1)

In [3]:
raw_data = pd.read_csv('data/train.csv')
target = torch.tensor(raw_data['label'].to_numpy(), dtype=torch.long)
data = torch.tensor(raw_data.iloc[:,1:].to_numpy().reshape((42000, 1, 28, 28)), dtype=torch.long) / 255.0

In [4]:
trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomAffine(degrees=(-10, 10), translate=(0, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor()
])

In [5]:
DATA = data
TARGET = target

len_data = len(data)
for step in range(25):
    new_data = torch.zeros((len_data, 1, 28, 28))
    for i, tensor in enumerate(data):
        new_data[i] = trans(tensor)
    DATA = torch.cat([DATA, new_data])
    TARGET = torch.cat([TARGET, target])
data = DATA
target = TARGET

In [6]:
n_epoch = 50
batch_size = 10000

networks = [Network_1().cuda()] + [Network_2().cuda()] + [Network_3().cuda()]
optimizers = [optim.Adam(Net.parameters(), lr=0.01) for Net in networks]
loss_fun = nn.CrossEntropyLoss().cuda()

In [7]:
def train(Net, X_train, y_train, optimizer, epoch):
    Net.train()
    for i in range(0, len(y_train), batch_size):
        X = X_train[i:i + batch_size].cuda()
        y = y_train[i:i + batch_size].cuda()

        def closure():
                optimizer.zero_grad()
                output = Net.forward(X)
                loss = loss_fun(output, y)
                loss.backward()   
                if i % 1000 == 0:
                        print('Train epoch: %s, {%s}, Loss: %s' % (epoch, i, loss.item()))
                return loss
        optimizer.step(closure)
        
def test(Net, X_test, y_test):
    test_loss = 0
    correct = 0

    Net.eval()
    with torch.no_grad():
        for i in range(len(X_test)):
            X = X_test[i:i + 1].cuda()
            y = y_test[i:i + 1].cuda()
            
            output = Net.forward(X)
            pred = output.argmax(dim=1)
#             if pred.item() != y.item():
#                 print(pred.item(), y.item(), i)
            correct += (pred == y).sum().item()
        print('TOTAL:', correct / len(X_test))
    

In [8]:
for i, network in enumerate(networks):
    indices = np.arange(0, len(data))
    np.random.shuffle(indices)
    X_train, y_train = data[indices], target[indices]
    
    print('TRAIN NETWORK #%s' % (i + 1))
    for epoch in range(n_epoch):
        train(network, X_train, y_train, optimizers[i], epoch)

TRAIN NETWORK #1
Train epoch: 0, {0}, Loss: 2.3100688457489014
Train epoch: 0, {10000}, Loss: 2.28869891166687
Train epoch: 0, {20000}, Loss: 2.2469704151153564
Train epoch: 0, {30000}, Loss: 2.1516058444976807
Train epoch: 0, {40000}, Loss: 2.008918046951294
Train epoch: 0, {50000}, Loss: 1.859309196472168
Train epoch: 0, {60000}, Loss: 1.6974976062774658
Train epoch: 0, {70000}, Loss: 1.5951411724090576
Train epoch: 0, {80000}, Loss: 1.468514084815979
Train epoch: 0, {90000}, Loss: 1.3580571413040161
Train epoch: 0, {100000}, Loss: 1.365869164466858
Train epoch: 0, {110000}, Loss: 1.274657964706421
Train epoch: 0, {120000}, Loss: 1.2164146900177002
Train epoch: 0, {130000}, Loss: 1.1749178171157837
Train epoch: 0, {140000}, Loss: 1.1113932132720947
Train epoch: 0, {150000}, Loss: 1.0956964492797852
Train epoch: 0, {160000}, Loss: 1.0218415260314941
Train epoch: 0, {170000}, Loss: 0.9838913083076477
Train epoch: 0, {180000}, Loss: 0.9593638777732849
Train epoch: 0, {190000}, Loss: 0.9

Train epoch: 1, {510000}, Loss: 0.3144257068634033
Train epoch: 1, {520000}, Loss: 0.3148210942745209
Train epoch: 1, {530000}, Loss: 0.2904805839061737
Train epoch: 1, {540000}, Loss: 0.3220061659812927
Train epoch: 1, {550000}, Loss: 0.32602164149284363
Train epoch: 1, {560000}, Loss: 0.3131103217601776
Train epoch: 1, {570000}, Loss: 0.30826103687286377
Train epoch: 1, {580000}, Loss: 0.3152896463871002
Train epoch: 1, {590000}, Loss: 0.3114810585975647
Train epoch: 1, {600000}, Loss: 0.29678601026535034
Train epoch: 1, {610000}, Loss: 0.3106194734573364
Train epoch: 1, {620000}, Loss: 0.32131871581077576
Train epoch: 1, {630000}, Loss: 0.3111228048801422
Train epoch: 1, {640000}, Loss: 0.2990928888320923
Train epoch: 1, {650000}, Loss: 0.30194345116615295
Train epoch: 1, {660000}, Loss: 0.31104719638824463
Train epoch: 1, {670000}, Loss: 0.28700047731399536
Train epoch: 1, {680000}, Loss: 0.29412293434143066
Train epoch: 1, {690000}, Loss: 0.2973344624042511
Train epoch: 1, {700000

Train epoch: 2, {1020000}, Loss: 0.25372833013534546
Train epoch: 2, {1030000}, Loss: 0.25298988819122314
Train epoch: 2, {1040000}, Loss: 0.2601899206638336
Train epoch: 2, {1050000}, Loss: 0.2527715563774109
Train epoch: 2, {1060000}, Loss: 0.260151207447052
Train epoch: 2, {1070000}, Loss: 0.26033294200897217
Train epoch: 2, {1080000}, Loss: 0.25450003147125244
Train epoch: 2, {1090000}, Loss: 0.26832225918769836
Train epoch: 3, {0}, Loss: 0.2577841877937317
Train epoch: 3, {10000}, Loss: 0.2586952745914459
Train epoch: 3, {20000}, Loss: 0.2657044231891632
Train epoch: 3, {30000}, Loss: 0.26592564582824707
Train epoch: 3, {40000}, Loss: 0.25287553668022156
Train epoch: 3, {50000}, Loss: 0.2655978500843048
Train epoch: 3, {60000}, Loss: 0.2416054904460907
Train epoch: 3, {70000}, Loss: 0.2589974105358124
Train epoch: 3, {80000}, Loss: 0.2685043513774872
Train epoch: 3, {90000}, Loss: 0.26146456599235535
Train epoch: 3, {100000}, Loss: 0.2749471664428711
Train epoch: 3, {110000}, Loss

Train epoch: 4, {420000}, Loss: 0.25689637660980225
Train epoch: 4, {430000}, Loss: 0.23648102581501007
Train epoch: 4, {440000}, Loss: 0.2439396232366562
Train epoch: 4, {450000}, Loss: 0.2342132329940796
Train epoch: 4, {460000}, Loss: 0.2530307471752167
Train epoch: 4, {470000}, Loss: 0.2404795140028
Train epoch: 4, {480000}, Loss: 0.24560639262199402
Train epoch: 4, {490000}, Loss: 0.2472846508026123
Train epoch: 4, {500000}, Loss: 0.24604442715644836
Train epoch: 4, {510000}, Loss: 0.23674596846103668
Train epoch: 4, {520000}, Loss: 0.2444310039281845
Train epoch: 4, {530000}, Loss: 0.23507104814052582
Train epoch: 4, {540000}, Loss: 0.24432194232940674
Train epoch: 4, {550000}, Loss: 0.24728144705295563
Train epoch: 4, {560000}, Loss: 0.2404659390449524
Train epoch: 4, {570000}, Loss: 0.23674722015857697
Train epoch: 4, {580000}, Loss: 0.2457817643880844
Train epoch: 4, {590000}, Loss: 0.24426832795143127
Train epoch: 4, {600000}, Loss: 0.22904643416404724
Train epoch: 4, {610000

Train epoch: 5, {910000}, Loss: 0.24366603791713715
Train epoch: 5, {920000}, Loss: 0.2207004874944687
Train epoch: 5, {930000}, Loss: 0.22709524631500244
Train epoch: 5, {940000}, Loss: 0.2272445559501648
Train epoch: 5, {950000}, Loss: 0.23055215179920197
Train epoch: 5, {960000}, Loss: 0.24257245659828186
Train epoch: 5, {970000}, Loss: 0.2213413566350937
Train epoch: 5, {980000}, Loss: 0.23374418914318085
Train epoch: 5, {990000}, Loss: 0.22867317497730255
Train epoch: 5, {1000000}, Loss: 0.22674082219600677
Train epoch: 5, {1010000}, Loss: 0.24164997041225433
Train epoch: 5, {1020000}, Loss: 0.22088703513145447
Train epoch: 5, {1030000}, Loss: 0.22793525457382202
Train epoch: 5, {1040000}, Loss: 0.2385721653699875
Train epoch: 5, {1050000}, Loss: 0.22586305439472198
Train epoch: 5, {1060000}, Loss: 0.232619047164917
Train epoch: 5, {1070000}, Loss: 0.22632569074630737
Train epoch: 5, {1080000}, Loss: 0.22823473811149597
Train epoch: 5, {1090000}, Loss: 0.2644227147102356
Train epo

...

Train epoch: 43, {520000}, Loss: 0.03396419808268547
Train epoch: 43, {530000}, Loss: 0.033652473241090775
Train epoch: 43, {540000}, Loss: 0.030299672856926918
Train epoch: 43, {550000}, Loss: 0.027658959850668907
Train epoch: 43, {560000}, Loss: 0.029511377215385437
Train epoch: 43, {570000}, Loss: 0.026272490620613098
Train epoch: 43, {580000}, Loss: 0.026187051087617874
Train epoch: 43, {590000}, Loss: 0.030732382088899612
Train epoch: 43, {600000}, Loss: 0.02327948622405529
Train epoch: 43, {610000}, Loss: 0.028387445956468582
Train epoch: 43, {620000}, Loss: 0.028338951990008354
Train epoch: 43, {630000}, Loss: 0.028724834322929382
Train epoch: 43, {640000}, Loss: 0.032890331000089645
Train epoch: 43, {650000}, Loss: 0.02713695354759693
Train epoch: 43, {660000}, Loss: 0.03332318738102913
Train epoch: 43, {670000}, Loss: 0.022063041105866432
Train epoch: 43, {680000}, Loss: 0.030489450320601463
Train epoch: 43, {690000}, Loss: 0.027913009747862816
Train epoch: 43, {700000}, Loss:

Train epoch: 44, {950000}, Loss: 0.028712108731269836
Train epoch: 44, {960000}, Loss: 0.030496424064040184
Train epoch: 44, {970000}, Loss: 0.028440892696380615
Train epoch: 44, {980000}, Loss: 0.02617221139371395
Train epoch: 44, {990000}, Loss: 0.028910059481859207
Train epoch: 44, {1000000}, Loss: 0.027466021478176117
Train epoch: 44, {1010000}, Loss: 0.03143701329827309
Train epoch: 44, {1020000}, Loss: 0.024938706308603287
Train epoch: 44, {1030000}, Loss: 0.026023434475064278
Train epoch: 44, {1040000}, Loss: 0.024253511801362038
Train epoch: 44, {1050000}, Loss: 0.028920313343405724
Train epoch: 44, {1060000}, Loss: 0.032590337097644806
Train epoch: 44, {1070000}, Loss: 0.022158388048410416
Train epoch: 44, {1080000}, Loss: 0.02641582116484642
Train epoch: 44, {1090000}, Loss: 0.00969446636736393
Train epoch: 45, {0}, Loss: 0.022780755534768105
Train epoch: 45, {10000}, Loss: 0.03162722662091255
Train epoch: 45, {20000}, Loss: 0.03969335928559303
Train epoch: 45, {30000}, Loss:

Train epoch: 46, {290000}, Loss: 0.02567957155406475
Train epoch: 46, {300000}, Loss: 0.025849096477031708
Train epoch: 46, {310000}, Loss: 0.027749627828598022
Train epoch: 46, {320000}, Loss: 0.028040502220392227
Train epoch: 46, {330000}, Loss: 0.02375086583197117
Train epoch: 46, {340000}, Loss: 0.028222637251019478
Train epoch: 46, {350000}, Loss: 0.027541181072592735
Train epoch: 46, {360000}, Loss: 0.025410715490579605
Train epoch: 46, {370000}, Loss: 0.021087484434247017
Train epoch: 46, {380000}, Loss: 0.02292393147945404
Train epoch: 46, {390000}, Loss: 0.03140883147716522
Train epoch: 46, {400000}, Loss: 0.03121413290500641
Train epoch: 46, {410000}, Loss: 0.02554929442703724
Train epoch: 46, {420000}, Loss: 0.026577521115541458
Train epoch: 46, {430000}, Loss: 0.0308526698499918
Train epoch: 46, {440000}, Loss: 0.025729339569807053
Train epoch: 46, {450000}, Loss: 0.031134573742747307
Train epoch: 46, {460000}, Loss: 0.026590881869196892
Train epoch: 46, {470000}, Loss: 0.0

Train epoch: 47, {730000}, Loss: 0.025753624737262726
Train epoch: 47, {740000}, Loss: 0.028735119849443436
Train epoch: 47, {750000}, Loss: 0.03797336667776108
Train epoch: 47, {760000}, Loss: 0.02383974939584732
Train epoch: 47, {770000}, Loss: 0.02972777932882309
Train epoch: 47, {780000}, Loss: 0.023596420884132385
Train epoch: 47, {790000}, Loss: 0.02619348093867302
Train epoch: 47, {800000}, Loss: 0.027715468779206276
Train epoch: 47, {810000}, Loss: 0.028587346896529198
Train epoch: 47, {820000}, Loss: 0.031551655381917953
Train epoch: 47, {830000}, Loss: 0.022642377763986588
Train epoch: 47, {840000}, Loss: 0.024982187896966934
Train epoch: 47, {850000}, Loss: 0.02681233547627926
Train epoch: 47, {860000}, Loss: 0.02717990055680275
Train epoch: 47, {870000}, Loss: 0.029180539771914482
Train epoch: 47, {880000}, Loss: 0.02602141723036766
Train epoch: 47, {890000}, Loss: 0.022843774408102036
Train epoch: 47, {900000}, Loss: 0.028531866148114204
Train epoch: 47, {910000}, Loss: 0.

Train epoch: 49, {60000}, Loss: 0.02941211871802807
Train epoch: 49, {70000}, Loss: 0.036687832325696945
Train epoch: 49, {80000}, Loss: 0.030627960339188576
Train epoch: 49, {90000}, Loss: 0.02796844393014908
Train epoch: 49, {100000}, Loss: 0.024296995252370834
Train epoch: 49, {110000}, Loss: 0.02899440936744213
Train epoch: 49, {120000}, Loss: 0.025238806381821632
Train epoch: 49, {130000}, Loss: 0.022773833945393562
Train epoch: 49, {140000}, Loss: 0.025304974988102913
Train epoch: 49, {150000}, Loss: 0.0244915634393692
Train epoch: 49, {160000}, Loss: 0.02866584248840809
Train epoch: 49, {170000}, Loss: 0.023923853412270546
Train epoch: 49, {180000}, Loss: 0.025860033929347992
Train epoch: 49, {190000}, Loss: 0.02087504416704178
Train epoch: 49, {200000}, Loss: 0.027830423787236214
Train epoch: 49, {210000}, Loss: 0.02348705753684044
Train epoch: 49, {220000}, Loss: 0.028321783989667892
Train epoch: 49, {230000}, Loss: 0.024673834443092346
Train epoch: 49, {240000}, Loss: 0.02566

In [15]:
torch.save(networks[0], 'model_1.pt')
torch.save(networks[1], 'model_2.pt')
torch.save(networks[2], 'model_3.pt')

In [36]:
test_data_csv = pd.read_csv('data/test.csv')
test_data = torch.tensor(test_data_csv.to_numpy().reshape((28000, 1, 28, 28)), dtype=torch.long) / 255.0

In [40]:
result = []
a = 10000
for i in range(0, len(test_data), a):
    output = networks[2].forward(test_data[i:i + a].cuda()).argmax(dim=1)
    result += [k.item() for k in output.cpu()]

In [64]:
ANS = pd.DataFrame({'ImageId': range(1, 28001), 'Label': result})
ANS = ANS.set_index('ImageId')
ANS.to_csv('result.csv')